In [20]:
import numpy as np
from functools import reduce
import plotly.graph_objects as go

import os
import sys
root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root)

from validation import get_error_propagation_prob, get_accuracy, get_precision, get_recall, get_f1
from src.scrapper import parse_conllu_file
from src.tagger import HiddenMarkovModel, HiddenMarkovModelTagger
# from src.visualization import plot_viterbi_path_binary, plot_viterbi_matrix

In [4]:
# Importing data
train = parse_conllu_file("../datasets/en_gum-ud-train.conllu") 
test = parse_conllu_file("../datasets/en_gum-ud-test.conllu")

In [5]:
# Training the HHM model:
tagger = HiddenMarkovModel(corpus=train).train()

In [8]:
# Evaluation of the model over the testing dataset:
test_predictions = tagger.predict(corpus=test)
print(f"""We have trained the model with {len(train)} sentences in english and the dataset contains {len(test)} sentences""")


We have trained the model with 8548 sentences in english and the dataset contains 1096 sentences



## Confussion Matrix

In [35]:
cm = tagger.get_confusion_matrix(corpus=test, corpus_prediction=test_predictions)

In [22]:
fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=tagger.tagset,
            y=tagger.tagset,
            colorscale="Thermal",
        )
    )
fig.show(legend=False)

As we can check, in general most of the predictions are good.
But the model have some problems detecting Nouns as Nouns, there are some case wich they are detected as PropN, Verb or Adj. Probably this is due to the dataset is unbalanced and there are more examples of Nouns than other tags.

We are going to go deeper in this information using the different metrics.

## Accuracy

In [34]:
acc = get_accuracy(cm)

print(f"Accuracy: {100*acc:.2f}%")

Accuracy: 82.13%


## Recall

In [47]:
print("Precision per tag:\n")

precs, p_preds, p_micro, p_macro = get_precision(cm)
for tag, precision, predictions in zip(tagger.tagset, precs, p_preds):
    print(f"{tag:<5} {100*precision:.2f}% over {int(predictions)} predictions.")

print(f"Micro precision: {100*p_micro:.2f} %")
print(f"Macro precision: {100*p_macro:.2f} %")
print()


Precision per tag:

noun  61.19% over 5377 predictions.
_     100.00% over 179 predictions.
adj   87.30% over 1008 predictions.
punct 99.14% over 2562 predictions.
det   96.40% over 1722 predictions.
verb  79.36% over 1899 predictions.
part  99.19% over 123 predictions.
cconj 99.13% over 686 predictions.
num   100.00% over 237 predictions.
sym   100.00% over 8 predictions.
pron  91.04% over 1551 predictions.
x     100.00% over 5 predictions.
intj  100.00% over 75 predictions.
sconj 97.62% over 84 predictions.
aux   83.54% over 972 predictions.
adp   82.61% over 2456 predictions.
adv   89.68% over 727 predictions.
propn 79.20% over 500 predictions.
Micro precision: 84.59 %
Macro precision: 91.41 %



## Probability of error acumulation
As you know the Hidden Markov Models are based on a first order approximation of the Markov's chain. This implies that the ith prediction is going to be dependant to the (i-1)th prediction.

That's why it could be interesting to develop a metric for measuring the probability of having 2 errors together, this metric measures la probabiliy of having a wrong prediction in the (i+1)th token if the previous prediction was wrong (ith).

In [15]:
error_probability = get_error_propagation_prob(test, test_predictions)
print(f"The probability of making a wrong prediction and then make another wrong prediction in the next token is {100*error_probability:.2f} %")

The probability of making a wrong prediction and then make another wrong prediction in the next token is 24.76 %


#### Let's check an example:
Imagine that we have the sentece "the blue cat" (Det Adj Noun)
And the model makes a wrong prediction for the first token ("The" = Noun).

The probability of having a wrong prediction for "blue" is 24.76%, and 75.24% of predicting "blue" as Adj.

This results are overall good if we have consider that the prediction is based on a wrong prediction.